In [ ]:
paraphrases = load("../data/llm-generated-pairs/paraphrases.json")
corpus = load("../data/corpus/corpus/corpus.jsonl")
model = "models/SPhilBERTa" 


In [2]:
def sample_negatives_irrelevant(paraphrases, corpus, model_path,
                                threshold=0.6, max_attempts=10):
 
    corpus_set = set(corpus)

    model = SentenceTransformer(model_path)
    negatives = []

    # for each pair identify the original sentence (i.e. contained in corpus file) 
    for pair in paraphrases:
        s1, s2 = pair

        if s1 in corpus_set:
            original = s1
        elif s2 in corpus_set:
            original = s2
        else:
            continue

        # for this sentence sample another random sentence from the corpus
        found = False
        for _ in range(max_attempts):
            candidate = random.choice(corpus)

            if candidate == original:
                continue

            # compute cosine similarity of the two sentences with the sentence transformer model 
            orig_emb = st_model.encode(original, convert_to_tensor=True)
            cand_emb = st_model.encode(candidate, convert_to_tensor=True)
            cosine_score = util.cos_sim(orig_emb, cand_emb).item()

            # if score < 0.6 append 
            if cosine_score < threshold:
                negatives.append((original, candidate))
                found = True
                break

    return negatives